In [ ]:
!nvidia-smi

Fri Apr 30 09:31:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq transformers datasets wandb 
    !pip install -Uqq fastcore onnx sentencepiece
    !pip install -Uqq --no-deps fastai
    !pip install -q git+git://github.com/aikindergarten/fasthugs.git
    !pip install -q git+git://github.com/aikindergarten/vat.git

In [ ]:
from fastai.text.all import *
from fastai.callback.wandb import *

from transformers import AutoModelForSequenceClassification
from fasthugs.learner import TransLearner
from fasthugs.data import TransformersTextBlock

from vat.core import ALUMCallback

In [ ]:
def read_text(fn):
    return open(fn).read()

In [ ]:
path = untar_data(URLs.IMDB)

## Setup

In [ ]:
ds_name = 'imdb'
model_name = 'microsoft/deberta-base'

max_len = 512
bs = 4
val_bs = bs*4

lr = 2e-5

## Tracking

In [ ]:
import wandb

WANDB_NAME = f'{ds_name}-{model_name}'
GROUP = f'{ds_name}-{model_name}-simple-{lr:.0e}'
NOTES = f'finetuning {model_name} with RAdam lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name, ds_name, 'radam', 'alum']

In [ ]:
wandb.init(reinit=True, project="vat", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)


## Training

In [ ]:
dblock = DataBlock(blocks = [TransformersTextBlock(pretrained_model_name=model_name), CategoryBlock()],
                   get_items=get_text_files,
                   get_x=read_text,
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='test'))

dls = dblock.dataloaders(path, bs=bs, val_bs=val_bs)
dls.show_batch(max_n=4)

,text,category
0,"Match 1: Tag Team Table Match Bubba Ray and Spike Dudley vs Eddie Guerrero and Chris Benoit Bubba Ray and Spike Dudley started things off with a Tag Team Table Match against Eddie Guerrero and Chris Benoit. According to the rules of the match, both opponents have to go through tables in order to get the win. Benoit and Guerrero heated up early on by taking turns hammering first Spike and then Bubba Ray. A German suplex by Benoit to Bubba took the wind out of the Dudley brother. Spike tried to help his brother, but the referee restrained him while Benoit and Guerrero ganged up on him in the corner. With Benoit stomping away on Bubba, Guerrero set up a table outside. Spike dashed into the ring and somersaulted over the top rope onto Guerrero on the outside! After recovering and taking care of Spike, Guerrero slipped a table",pos
1,"""La Maman et la putain"" is the beautifulest film of all time. And what's most moving about it may be the relation between reality and art the movie deals with, which is directly inspired by Proust's ""A la Recherche du temps perdu"".<br /><br />Indeed, ""La Maman et la putain"" and ""In search of lost time"" apparently tell the same story : the one of the failure of love, which repeats itself endlessly. The first woman's name is always Gilberte, and the second woman appears like a twisted and deformed double of Gilberte : Veronika is like a ""whore Gilberte"", beautiful like the night, whereas Gilberte was pure, and ""beautiful like the day"". After the failure of the first love, a second love begins, but this one is like already doomed by the first one. Veronika takes the place of Gilberte, in Alexandre's life and in the movie. She progressively eclipses",pos
2,"I watched Grendel the other night and am compelled to put together a Public Service Announcement.<br /><br />Grendel is another version of Beowulf, the thousand-year-old Anglo-Saxon epic poem. The SciFi channel has a growing catalog of inoffensive and uninteresting movies, and the previews promised an inauthentic low-budget mini-epic, but this one refused to let me switch channels. It was staggeringly, overwhelmingly, bad. I watched in fascination and horror at the train wreck you couldn't tear your eyes away from. I reached for a notepad and managed to capture part of what I was seeing. The following may contain spoilers or might just save your sanity. You've been warned.<br /><br />- Just to get it over with, Beowulf's warriors wore horned helmets. Trivial issue compared to what came after. It also appears that the helmets were in a bin and handed to whichever actor wandered by next. Fit, appearance and function",neg
3,"This project was originally conceived as the movie version of popular Japanese manga SlamDunk! and that's not something new to Jay Chou, who made his movie debut playing a character from another wildly popular manga Initial D. Along the way, it was decided to incorporate some kung fu into the movie, so hence the title, even if the idea wasn't very original, with Stephen Chow's Shaolin Soccer coming to mind with martial arts and ball games combined.<br /><br />However, and thankfully, those scenes where kung fu actually influenced the games were kept to a bare minimum, and in Kung Fu Dunk, really quite unnecessary, because they don't add much to the plot nor drum up much excitement, and at most offered some cheap laughs and reminisced about the time when Stephen Chow used kung fu in football games. Jay Chou is comfortable in his role as a martial artist Fang",pos


### ALUM finetuning

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
learn = TransLearner(dls, model,
                opt_func=RAdam,
                cbs=[GradientAccumulation(16)],
                metrics=[accuracy]).to_fp16()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight

In [ ]:
learn.add_cb(ALUMCallback(learn.model.base_model.embeddings, start_epoch=0, alpha=0.7));

In [ ]:
learn.fit_one_cycle(4, 2e-5, cbs=WandbCallback(log_preds=False, log_model=False))

Could not gather input dimensions


epoch,train_loss,valid_loss,accuracy,time
0,0.328478,0.227183,0.947080,1:11:19
1,0.098229,0.117810,0.956760,1:11:46
2,0.055866,0.112570,0.959520,1:11:29
3,0.031281,0.115716,0.960280,1:11:25


Starting virtual adversarial training at epoch 0


Exception ignored in: <finalize object at 0x7fa510d4c930; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.py", line 483, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpuqd1tkhg'
Exception ignored in: <finalize object at 0x7fa510d4c930; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
    _rmtree(name)
  File "/usr/lib/python3.7/shutil.py", line 485, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/usr/lib/python3.7/shutil.p